In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sb
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.preprocessing
import scipy.cluster.hierarchy as sch
import skfuzzy as fuzz
import pylab
import sklearn.mixture as mixture
import pyclustertend 
import random
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

Variables:
- Id: Id de la película
- popularity: Índice de popularidad de la película calculado semanalmente
- budget: El presupuesto para la película.
- revenue: El ingreso de la película.
- original_title: El título original de la película, en su idioma original.
- originalLanguage: Idioma original en que se encuentra la película
- title: El título de la película traducido al inglés
- homePage: La página de inicio de la película
- video: Si tiene videos promocionales o no
- director: Director de la película
- runtime: La duración de la película.
- genres: El género de la película.
- genresAmount: Cantidad de géneros que representan la película
- productionCompany: Las compañías productoras de la película.
- productionCoAmount: Cantidad de compañías productoras que participaron en la película
- productionCompanyCountry: Países de las compañías productoras de la película
- productionCountry: Países en los que se llevó a cabo la producción de la película
- productionCountriesAmount: Cantidad de países en los que se rodó la película
- releaseDate: Fecha de lanzamiento de la película
- voteCount: El número de votos en la plataforma para la película.
- voteAvg: El promedio de los votos en la plataforma para la película
- actors: Actores que participan en la película (Elenco)
- actorsPopularity: Índice de popularidad del elenco de la película.
- actorsCharacter: Personaje que interpreta cada actor en la película
- actorsAmount: Cantidad de personas que actúan en la película
- castWomenAmount: Cantidad de actrices en el elenco de la película
- castMenAmount: Cantidad de actores en el elenco de la película.

In [3]:
# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
# Configuración para mostrar todas las filas
pd.set_option('display.max_rows', None)
archivo_csv = "movies.csv"
datos = pd.read_csv(archivo_csv, encoding='ISO-8859-1')

id
5         1
6         1
11        1
12        1
13        1
14        1
15        1
16        1
18        1
19        1
22        1
24        1
25        1
27        1
28        1
33        1
35        1
38        1
55        1
58        1
59        1
62        1
63        1
65        1
66        1
68        1
69        1
70        1
71        1
73        1
74        1
75        1
76        1
77        1
78        1
79        1
80        1
81        1
82        1
83        1
85        1
87        1
88        1
89        1
90        1
93        1
95        1
96        1
97        1
98        1
101       1
103       1
104       1
105       1
106       1
107       1
108       1
109       1
110       1
111       1
113       1
114       1
115       1
116       1
117       1
118       1
120       1
121       1
122       1
123       1
128       1
129       1
134       1
136       1
137       1
138       1
141       1
142       1
143       1
144       1
146       1
147       1
149       1
1

In [4]:
print(datos.isnull().sum())  # Cantidad de valores nulos por columna
print(datos.duplicated().sum())  # Cantidad de filas duplicadas


id                              0
budget                          0
genres                         53
homePage                     5807
productionCompany             457
productionCompanyCountry     1280
productionCountry             233
revenue                         0
runtime                         0
video                         486
director                       74
actors                         80
actorsPopularity               87
actorsCharacter                47
originalTitle                   0
title                           0
originalLanguage                0
popularity                      0
releaseDate                     0
voteAvg                         0
voteCount                       0
genresAmount                    0
productionCoAmount              0
productionCountriesAmount       0
actorsAmount                    0
castWomenAmount                 0
castMenAmount                   0
dtype: int64
0


In [5]:
datos = datos.drop_duplicates()


Columna con valores no numéricos: genres
0                        Crime|Comedy
1               Action|Thriller|Crime
2    Adventure|Action|Science Fiction
3                    Animation|Family
4                Comedy|Drama|Romance
Name: genres, dtype: object
Columna con valores no numéricos: actorsPopularity
0    22.225|23.519|17.816|19.893|9.027|7.147|7.769|...
1    9.008|6.383|10.757|18.295|11.772|14.777|9.669|...
2    11.881|24.542|14.434|10.651|6.888|5.811|1.432|...
3    9.79|8.084|8.538|33.379|11.733|11.866|8.141|12...
4    33.297|13.246|6.748|6.402|13.189|10.974|10.259...
Name: actorsPopularity, dtype: object
Columna con valores no numéricos: originalLanguage
0    en
1    en
2    en
3    en
4    en
Name: originalLanguage, dtype: object
Columna con valores no numéricos: releaseDate
0    1995-12-09
1    1993-10-15
2    1977-05-25
3    2003-05-30
4    1994-07-06
Name: releaseDate, dtype: object
Columna con valores no numéricos: castWomenAmount
0    15
1     3
2     5
3     5
4    18

ValueError: could not convert string to float: 'Crime'

ValueError: could not convert string to float: '22.225|23.519|17.816|19.893|9.027|7.147|7.769|6.476|7.906|12.838|17.988|20.027|4.47|0.6|1.286|23.95|22.082|3.44|27.973|19.062|2.688|2.079|1.008|7.647|0.6'